## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import ipywidgets as widgets

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.20,65,31,20.31,scattered clouds
1,1,Nikolskoye,RU,59.7035,30.7861,37.87,54,100,7.65,overcast clouds
2,2,Atasu,KZ,48.6814,71.6433,55.62,40,98,7.90,overcast clouds
3,3,Stoughton,US,42.1251,-71.1023,52.56,36,89,7.00,overcast clouds
4,4,Manta,EC,-0.9500,-80.7333,82.18,74,40,5.75,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.20,65,31,20.31,scattered clouds
4,4,Manta,EC,-0.9500,-80.7333,82.18,74,40,5.75,scattered clouds
13,13,Atuona,PF,-9.8000,-139.0333,78.78,78,57,19.62,light rain
21,21,Hilo,US,19.7297,-155.0900,85.28,75,75,8.05,broken clouds
23,23,Kattivakkam,IN,13.2167,80.3167,85.06,89,19,14.94,few clouds
...,...,...,...,...,...,...,...,...,...,...
692,692,Ternate,ID,0.8000,127.4000,80.46,78,95,2.17,overcast clouds
694,694,Mareeba,AU,-17.0000,145.4333,75.43,86,100,2.44,overcast clouds
697,697,Tabas,IR,33.5959,56.9244,80.82,21,79,2.30,broken clouds
701,701,Ascension,US,30.2169,-90.9001,84.06,32,0,6.91,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.20,65,31,20.31,scattered clouds
4,4,Manta,EC,-0.9500,-80.7333,82.18,74,40,5.75,scattered clouds
13,13,Atuona,PF,-9.8000,-139.0333,78.78,78,57,19.62,light rain
21,21,Hilo,US,19.7297,-155.0900,85.28,75,75,8.05,broken clouds
23,23,Kattivakkam,IN,13.2167,80.3167,85.06,89,19,14.94,few clouds
...,...,...,...,...,...,...,...,...,...,...
692,692,Ternate,ID,0.8000,127.4000,80.46,78,95,2.17,overcast clouds
694,694,Mareeba,AU,-17.0000,145.4333,75.43,86,100,2.44,overcast clouds
697,697,Tabas,IR,33.5959,56.9244,80.82,21,79,2.30,broken clouds
701,701,Ascension,US,30.2169,-90.9001,84.06,32,0,6.91,clear sky


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.20,scattered clouds,-23.1203,-134.9692,
4,Manta,EC,82.18,scattered clouds,-0.9500,-80.7333,
13,Atuona,PF,78.78,light rain,-9.8000,-139.0333,
21,Hilo,US,85.28,broken clouds,19.7297,-155.0900,
23,Kattivakkam,IN,85.06,few clouds,13.2167,80.3167,
24,Kahului,US,87.82,scattered clouds,20.8947,-156.4700,
27,Garowe,SO,77.86,scattered clouds,8.4054,48.4845,
28,Faanui,PF,82.29,clear sky,-16.4833,-151.7500,
30,Kutum,SD,79.75,overcast clouds,14.2000,24.6667,
31,Rayagada,IN,80.55,clear sky,19.1667,83.4167,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        clean_hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
        

In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.20,scattered clouds,-23.1203,-134.9692,People ThankYou
4,Manta,EC,82.18,scattered clouds,-0.9500,-80.7333,Balandra Hotel
13,Atuona,PF,78.78,light rain,-9.8000,-139.0333,Villa Enata
21,Hilo,US,85.28,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
23,Kattivakkam,IN,85.06,few clouds,13.2167,80.3167,Euronova Freezer
...,...,...,...,...,...,...,...
692,Ternate,ID,80.46,overcast clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
694,Mareeba,AU,75.43,overcast clouds,-17.0000,145.4333,Trinity Plains Tourist Park
697,Tabas,IR,80.82,broken clouds,33.5959,56.9244,شرکت معماری مهراز شایگان آراد
701,Ascension,US,84.06,clear sky,30.2169,-90.9001,Highland Inn


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
<dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))